# Import

In [63]:
import nltk
import pandas as pd
import numpy as np

# Installlation

In [64]:
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('brown')
# nltk.download('tagsets')
# nltk.download('gutenberg')
# nltk.download('genesis')
# nltk.download('inaugural')
# nltk.download('nps_chat')

# Load dataset

In [65]:
file = open('nltk.txt')
read_file = file.read()
text = nltk.Text(nltk.word_tokenize(read_file))
tags = nltk.pos_tag(text)
words = [word for (word, pos) in nltk.pos_tag(text)]
pos = [pos for (word, pos) in nltk.pos_tag(text)]
# match = text.concordance('car')

In [66]:
prh = {'Words':words,'Pos':pos}
df = pd.DataFrame(prh)
df

,Words,Pos
0,.,.
1,The,DT
2,is,VBZ
3,a,DT
4,car,NN
5,.,.
6,Will,NNP
7,is,VBZ
8,running,VBG
9,.,.


## Adding sentence id

In [67]:
senDf = df.copy()
senDf

,Words,Pos
0,.,.
1,The,DT
2,is,VBZ
3,a,DT
4,car,NN
5,.,.
6,Will,NNP
7,is,VBZ
8,running,VBG
9,.,.


In [68]:
#All examples position
indPos = senDf[senDf['Words']=="."]
indPos

,Words,Pos
0,.,.
5,.,.
9,.,.
14,.,.
20,.,.


# Input

In [69]:
word1 = "VBG"
word2 = "DT"

# Filtering

### Filter 1

In [70]:
res = df[(df['Words'].str.contains(word1)) | (df['Pos'].str.contains(word1))]
res['index'] = res.index
res

<ipython-input-70-80d80e85ee86>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['index'] = res.index


,Words,Pos,index
8,running,VBG,8
17,taking,VBG,17


In [71]:
# first filter passed
indices = res.iloc[:,2].values
indices

array([ 8, 17], dtype=int64)

### Filter 2

In [72]:
#Input: the index of the searched word
# Output: the position of the end of the example
def upperIndex(item,alist):
    #find the greater border index 
    position = np.argmax(alist > item)
    return generalIndex[position]

In [73]:
r = upperIndex(17,indPos.index)
r

20

In [62]:
#Get the index of the good example
examplesPosition = []
for i in indices:
    endExample = upperIndex(i,indPos.index)
    if(i+1<endExample):
        print(endExample)
        #Two verifiy that the searched word exist in the 3 next words 
        if((df.iloc[i+1].Words.lower() == word2.lower() or df.iloc[i+1].Pos.lower() == word2.lower()) or
          i+2<endExample and ((df.iloc[i+2].Words.lower() == word2.lower() or df.iloc[i+2].Pos.lower() == word2.lower())) or
          i+3<endExample and ((df.iloc[i+3].Words.lower() == word2.lower() or df.iloc[i+3].Pos.lower() == word2.lower()))):
                examplesPosition.append(i)
examplesPosition

20


[17]

# Result

## Get the good examples indices

In [29]:
examplesPosition

[8, 17]

In [30]:
generalIndex = indPos.index.array
generalIndex

<PandasArray>
[0, 5, 9, 14, 20]
Length: 5, dtype: int64

In [31]:
#Find the list of index of the final example 
finalIndexList = []
for i in examplesPosition:
    #find the greater border index 
    position = np.argmax(generalIndex > i)
    finalIndexList.append(generalIndex[position])
    
    #Get the smaller border index
    indexes = [j for j, x in enumerate(generalIndex) if x < i]
    finalIndexList.append(generalIndex[max(indexes)])
    
finalIndexList.sort()
finalIndexList

[5, 9, 14, 20]

## Create the examples pairs

In [32]:
splitedSize = 2
examplesIndex = [finalIndexList[x:x+splitedSize] for x in range(0, len(finalIndexList), splitedSize)]

In [33]:
examplesIndex

[[5, 9], [14, 20]]

## Create the dataframe of the solution

In [74]:
# Create the solution dataframe
columns = ['Words','Pos']
finalSolution = pd.DataFrame(columns=columns)

for exa in examplesIndex:
    finalSolution = finalSolution.append(df.iloc[exa[0]:exa[1]])
finalSolution

,Words,Pos
5,.,.
6,Will,NNP
7,is,VBZ
8,running,VBG
14,.,.
15,I,PRP
16,'m,VBP
17,taking,VBG
18,the,DT
19,plane,NN


In [35]:
[finalSolution.Words]

[5           .
 6        Will
 7          is
 8     running
 14          .
 15          I
 16         'm
 17     taking
 18        the
 19      plane
 Name: Words, dtype: object]

# Show the final answer